In [1]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
print ('import statent grid')
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)

print('read statpop that has been pre-joined with statent')
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
)

import statent grid
read statpop that has been pre-joined with statent


In [9]:
print('read transport network default')
transport_network_default = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip')
)

read transport network default


In [4]:
print('read transport network cycling')
transport_network_cycling = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_cycling.osm.pbf'),
)
print('read transport network car')
transport_network_car = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_car.osm.pbf'),
)

read transport network cycling


In [6]:
perimeter = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), index='id'
).geometry['ebc_zrh_v01'].simplify(500, preserve_topology=True)
statent_in_perimeter = statent[statent.within(perimeter)]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

cells = list(statent_in_perimeter['id'].sample(n=1500, random_state=0))

def miniwrapper(cell):
    print(f'calculating cell {cell}')
    return snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_default,
        transport_network_car,
        transport_network_cycling,
        statpop, statent, cell,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.01
    )

accessibility = pd.concat(
    map(miniwrapper, cells),
    ignore_index=True
)

print('exporting file')
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, 'accessibility.gpkg')
)

calculating cell 70052597
calculating cell 67152430
calculating cell 69622454
calculating cell 68552443
calculating cell 68282638
calculating cell 69762622
calculating cell 68422479
calculating cell 68802425
calculating cell 68402732
calculating cell 69362305
calculating cell 68252501
calculating cell 69372359
calculating cell 69272485
calculating cell 68642455
calculating cell 68272741
calculating cell 68292379
calculating cell 70542272
calculating cell 68102469
calculating cell 69032459
calculating cell 68792351
calculating cell 68312524
calculating cell 69542341
calculating cell 67732504
calculating cell 69702619
calculating cell 70212412
calculating cell 69222500
calculating cell 69292579
